In [16]:
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.core.debugger import set_trace


In [18]:
import numpy

def convert_to_gb(string):
    conversion = {'T':1024.,'G':1.,'M':1./1024.,'K':1./1024./1024}
    string_updated = str(string)
    stem = string_updated[:-1]
    suffix = string_updated[-1]
    conversion_factor = conversion[suffix]
    try:
        number_of_gb = conversion_factor * float(stem)
    except:
        number_of_gb = numpy.nan
    return number_of_gb



def sge_qacct_parse(filename):
    # Import libraries
    import pandas as pd
    
    # Parse qacct log file into dictionary 
    dicts=[]
    with open(filename, 'r') as f: 
        d = {}
        for line in f:
            # set_trace()
            if line[0:3] == "===" or line[0:18] == 'Total System Usage':
                dicts.append(d)
                d = {}
                continue
            if line[0:13] == "    WALLCLOCK":
                break
            field = line[0:13].strip()
            value = line[13:-1].strip()
            if value == "8990":
                set_trace()
            d[field] = value
        dicts.append(d)
    # Remove first element in dicts list, which is empty
    del dicts[0]
    
    
    
    # Load data into dataframe and set correct types
    df = pd.DataFrame(dicts)
    # set_trace()
    floatfields = ['priority',
    'ru_wallclock',
    'ru_utime',
    'ru_stime',
    'ru_maxrss',
    'ru_ixrss',
    'ru_ismrss',
    'ru_idrss',
    'ru_isrss',
    'ru_minflt',
    'ru_majflt',
    'ru_nswap',
    'ru_inblock',
    'ru_oublock',
    'ru_msgsnd',
    'ru_msgrcv',
    'ru_nsignals',
    'ru_nvcsw',
    'ru_nivcsw',
    'cpu',
    'mem',
    'io',
    'iow']
    for field in floatfields: 
            df[field] = df[field].astype('float')
    
    df["maxvmem"] = df["maxvmem"].apply(lambda s: convert_to_gb(s) )
    df["maxvmem"] = df["maxvmem"].astype('float')
    datefields = ['qsub_time','start_time','end_time']
    for field in datefields: 
        df[field] = pd.to_datetime(df[field])  #,format='%m/%d/%y %I:%M%p'
    intfields = ['slots','jobnumber','taskid','exit_status'] 
    for field in intfields: 
        df[field] = df[field].astype('int')

    return df



In [19]:
# sge_qacct_parse("/Users/jessegarcia/sge_tools/qacct_output.txt")
test = sge_qacct_parse("hoffman2_qacct_output.txt")



> /var/folders/4q/5rvz5xqd0_d2lv800sc0qpp80000gn/T/ipykernel_58501/2843893668.py(37)sge_qacct_parse()
     35             if value == "8990":
     36                 set_trace()
---> 37             d[field] = value
     38     # Remove first element in dicts list, which is empty
     39     del dicts[0]

'8990'
*** Newest frame
> /var/folders/4q/5rvz5xqd0_d2lv800sc0qpp80000gn/T/ipykernel_58501/2843893668.py(25)sge_qacct_parse()
     23     with open(filename, 'r') as f:
     24         d = {}
---> 25         for line in f:
     26             # set_trace()
     27             if line[0:3] == "===" or line[0:18] == 'Total System Usage':

> /var/folders/4q/5rvz5xqd0_d2lv800sc0qpp80000gn/T/ipykernel_58501/2843893668.py(27)sge_qacct_parse()
     25         for line in f:
     26             # set_trace()
---> 27             if line[0:3] == "===" or line[0:18] == 'Total System Usage':
     28                 dicts.append(d)
     29                 d = {}

> /var/folders/4q/5rvz5xqd0_d2lv800

BdbQuit: 

In [13]:
test[test["taskid"] == 8990]


,qname,hostname,group,owner,project,department,jobname,jobnumber,taskid,pe_taskid,account,priority,cwd,submit_host,submit_cmd,qsub_time,start_time,end_time,granted_pe,slots,failed,deleted_by,exit_status,ru_wallclock,ru_utime,ru_stime,ru_maxrss,ru_ixrss,ru_ismrss,ru_idrss,ru_isrss,ru_minflt,ru_majflt,ru_nswap,ru_inblock,ru_oublock,ru_msgsnd,ru_msgrcv,ru_nsignals,ru_nvcsw,ru_nivcsw,wallclock,cpu,mem,io,iow,ioops,maxvmem,maxrss,maxpss,arid,jc_name,bound_cores


In [68]:
failed = test[test["exit_status"] != 0]
failed.cpu
failed.columns
failed.exit_status

failed[failed["exit_status"] == 1]

,qname,hostname,group,owner,project,department,jobname,jobnumber,taskid,pe_taskid,account,priority,cwd,submit_host,submit_cmd,qsub_time,start_time,end_time,granted_pe,slots,failed,deleted_by,exit_status,ru_wallclock,ru_utime,ru_stime,ru_maxrss,ru_ixrss,ru_ismrss,ru_idrss,ru_isrss,ru_minflt,ru_majflt,ru_nswap,ru_inblock,ru_oublock,ru_msgsnd,ru_msgrcv,ru_nsignals,ru_nvcsw,ru_nivcsw,wallclock,cpu,mem,io,iow,ioops,maxvmem,maxrss,maxpss,arid,jc_name,bound_cores
1641,pod_smp.q,n6019,eeskin,jessegar,eeskin_prj,defaultdepartment,model4,1814430,1769,NONE,sge,0.0,/u/project/klohmuel/jessegar/dfe_inference/cod...,login4,qsub jobwrapper_run_simulation_feb_23_2022.sh,2022-02-23 18:18:16.534,2022-02-24 04:55:37.736,2022-02-24 04:55:44.226,NONE,1,0,NONE,1,6.49,5.01,3.385,131572.0,0.0,0.0,0.0,0.0,288527.0,0.0,0.0,848.0,496.0,0.0,0.0,0.0,12285.0,435682.0,6.743,8.394,1.28,0.058,0.0,13944,3.179,0.000,0.000,undefined,NONE,NONE
